# XGBoost Classifier (Gradient-Boosting-Verfahren)

Imports

In [17]:
import time
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import numpy as np
import joblib

import xgboost as xgb

Config for XGBoost

In [18]:
pre_dispatch_jobs = 12
useGpu = False

Load data and labels into dataframes

In [19]:
df = pd.read_csv("./data/lucas_organic_carbon_training_and_test_data_NEW.csv")
labels = pd.read_csv("./data/lucas_organic_carbon_target.csv")

In [31]:
df.describe()

,500.0,500.5,501.0,501.5,502.0,502.5,503.0,503.5,504.0,504.5,...,2495.0,2495.5,2496.0,2496.5,2497.0,2497.5,2498.0,2498.5,2499.0,2499.5
count,9895.000000,9895.000000,9895.000000,9895.000000,9895.000000,9895.000000,9895.000000,9895.000000,9895.000000,9895.000000,...,9895.000000,9895.000000,9895.000000,9895.000000,9895.000000,9895.000000,9895.000000,9895.000000,9895.000000,9895.000000
mean,0.000318,0.000320,0.000322,0.000323,0.000325,0.000327,0.000329,0.000331,0.000333,0.000335,...,-0.000085,-0.000082,-0.000079,-0.000076,-0.000073,-0.000070,-0.000067,-0.000064,-0.000061,-0.000058
std,0.000130,0.000131,0.000132,0.000132,0.000133,0.000134,0.000134,0.000135,0.000136,0.000136,...,0.000047,0.000046,0.000046,0.000045,0.000045,0.000044,0.000044,0.000044,0.000043,0.000043
min,0.000016,0.000016,0.000016,0.000017,0.000017,0.000017,0.000017,0.000018,0.000018,0.000018,...,-0.000558,-0.000557,-0.000556,-0.000555,-0.000554,-0.000554,-0.000553,-0.000552,-0.000551,-0.000550
25%,0.000221,0.000223,0.000224,0.000226,0.000227,0.000229,0.000230,0.000231,0.000233,0.000234,...,-0.000110,-0.000107,-0.000104,-0.000100,-0.000097,-0.000094,-0.000091,-0.000088,-0.000085,-0.000082
50%,0.000310,0.000312,0.000314,0.000316,0.000318,0.000320,0.000321,0.000323,0.000325,0.000327,...,-0.000077,-0.000074,-0.000071,-0.000068,-0.000065,-0.000062,-0.000059,-0.000056,-0.000053,-0.000050
75%,0.000408,0.000410,0.000412,0.000415,0.000417,0.000419,0.000422,0.000424,0.000426,0.000429,...,-0.000051,-0.000049,-0.000046,-0.000043,-0.000040,-0.000038,-0.000035,-0.000032,-0.000029,-0.000026
max,0.000876,0.000881,0.000887,0.000892,0.000897,0.000902,0.000907,0.000912,0.000917,0.000923,...,0.000033,0.000035,0.000039,0.000043,0.000047,0.000051,0.000055,0.000060,0.000064,0.000068


Create Scaler and scala data

In [21]:
# TODO: Maybe try a z-transfromtion stead of the standardscaler
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df)
scaled_df = pd.DataFrame(scaled_data, columns=df.columns)
scaled_df

,500.0,500.5,501.0,501.5,502.0,502.5,503.0,503.5,504.0,504.5,...,2495.0,2495.5,2496.0,2496.5,2497.0,2497.5,2498.0,2498.5,2499.0,2499.5
0,-0.317694,-0.318224,-0.318744,-0.319256,-0.319759,-0.320253,-0.320739,-0.321217,-0.321686,-0.322148,...,0.320884,0.311196,0.301124,0.290665,0.279820,0.268590,0.256980,0.244997,0.232651,0.219954
1,0.256876,0.254926,0.252993,0.251076,0.249174,0.247289,0.245420,0.243566,0.241727,0.239904,...,0.953367,0.949280,0.944598,0.939293,0.933338,0.926704,0.919368,0.911309,0.902508,0.892950
2,0.091526,0.090582,0.089648,0.088721,0.087803,0.086892,0.085990,0.085095,0.084209,0.083330,...,-0.016399,-0.031121,-0.046161,-0.061504,-0.077135,-0.093032,-0.109173,-0.125532,-0.142080,-0.158787
3,-0.132203,-0.129195,-0.126215,-0.123263,-0.120339,-0.117442,-0.114573,-0.111730,-0.108914,-0.106124,...,-0.503614,-0.486400,-0.468538,-0.450026,-0.430868,-0.411068,-0.390637,-0.369589,-0.347943,-0.325724
4,0.536071,0.533381,0.530712,0.528063,0.525434,0.522825,0.520236,0.517667,0.515117,0.512587,...,0.194378,0.177311,0.159763,0.141743,0.123262,0.104338,0.084991,0.065242,0.045121,0.024658
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9890,1.068931,1.065211,1.061515,1.057844,1.054196,1.050572,1.046972,1.043396,1.039844,1.036315,...,-1.896294,-1.884456,-1.871355,-1.856935,-1.841145,-1.823937,-1.805269,-1.785104,-1.763413,-1.740174
9891,-0.533231,-0.528060,-0.522935,-0.517854,-0.512817,-0.507825,-0.502875,-0.497969,-0.493105,-0.488283,...,0.317374,0.315407,0.313229,0.310831,0.308203,0.305338,0.302229,0.298870,0.295256,0.291383
9892,0.557028,0.562643,0.568195,0.573686,0.579116,0.584485,0.589796,0.595047,0.600241,0.605379,...,-1.968334,-1.971239,-1.973170,-1.974056,-1.973825,-1.972409,-1.969740,-1.965754,-1.960391,-1.953597
9893,0.121474,0.111586,0.101796,0.092101,0.082501,0.072994,0.063580,0.054258,0.045026,0.035884,...,1.528755,1.514539,1.499200,1.482700,1.465000,1.446070,1.425882,1.404414,1.381652,1.357588


Use lable encoder to transform high, low, etc to 1, 2, 3, 4, 5

In [22]:
label_encoder = LabelEncoder()
labels.x = label_encoder.fit_transform(labels.x)

Split Dataset into Training- and Testset

In [23]:
X_train, X_test, y_train, y_test = train_test_split(scaled_data, labels, test_size=0.2, random_state=42)

Create XGBoost Classifier

In [24]:
if useGpu:
    clf = xgb.XGBClassifier(learning_rate=0.02, n_estimators=10, objective='multi:softmax',
                            num_class=len(pd.unique(labels.x)), tree_method = "hist", device = "cuda")
else:
    clf = xgb.XGBClassifier(learning_rate=0.02, n_estimators=10, objective='multi:softmax',
                            num_class=len(pd.unique(labels.x)))
    
clf

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.02, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=10, n_jobs=None, num_class=5,
              num_parallel_tree=None, ...)

Create Parametergrid for Gridsearch / Randomsearch

In [25]:
param_grid = {
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [1.0, 1.2],
    'gamma': [1],
    'min_child_weight': [5, 10],
    'max_depth': [3],
    'n_estimators': [int(x) for x in np.linspace(start = 550, stop = 650, num = 4)],
    'learning_rate': [0.1, 0.05, 0.15]
}
param_grid

{'subsample': [0.6, 0.8, 1.0],
 'colsample_bytree': [1.0, 1.2],
 'gamma': [1],
 'min_child_weight': [5, 10],
 'max_depth': [3],
 'n_estimators': [550, 583, 616, 650],
 'learning_rate': [0.1, 0.05, 0.15]}

Create Scoring method

In [26]:
scorer = make_scorer(f1_score, average='micro')

Run Grid/Randomsearch

In [27]:
grid_search = GridSearchCV(clf, param_grid, cv=5, refit=True, verbose=3, n_jobs=-1,
                    pre_dispatch=pre_dispatch_jobs)

fit_time = time.perf_counter()
grid_search.fit(X_train, y_train)
fit_time = np.round(time.perf_counter() - fit_time)
print(f"Total Time to fit {fit_time}")

Fitting 5 folds for each of 144 candidates, totalling 720 fits


KeyboardInterrupt: 

Extract the best classifier, its paramester and the score

In [ ]:
best_clf = grid_search.best_estimator_
best_params = grid_search.best_params_
best_score = grid_search.best_score_

Run predictions on test-data and show model metrics

In [ ]:
# Make predictions on the test set
y_pred = best_clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Save model

In [ ]:
joblib.dump(best_clf, './models/xgboost.pkl')

Load model

In [28]:
model = joblib.load('./models/best_model_xgboost.pkl')
model

/Users/moritzlindner/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [13:27:39] WARNING: /Users/runner/work/xgboost/xgboost/src/gbm/gbtree.cc:385: Changing updater from `grow_gpu_hist` to `grow_quantile_histmaker`.
  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=1.0, device='cuda', early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=1, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=550, n_jobs=None, num_class=5,
              num_parallel_tree=None, ...)

In [29]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.7887822132390097
Classification Report:
              precision    recall  f1-score   support

           0       0.60      0.59      0.59       130
           1       0.36      0.14      0.21       224
           2       0.45      0.38      0.41       190
           3       0.65      0.54      0.59        28
           4       0.87      0.97      0.91      1407

    accuracy                           0.79      1979
   macro avg       0.58      0.52      0.54      1979
weighted avg       0.75      0.79      0.76      1979

Confusion Matrix:
[[  77    7   31    7    8]
 [   6   32   38    0  148]
 [  30   30   72    1   57]
 [  11    0    2   15    0]
 [   5   19   18    0 1365]]


In [32]:
X_test.describe()

AttributeError: 'numpy.ndarray' object has no attribute 'describe'

In [30]:
pd.unique(y_pred)

array([4, 2, 3, 1, 0], dtype=int32)

In [ ]:
f1 = f1_score(y_test, y_pred, average='weighted')
f1